In [0]:
import pyspark

In [0]:
sdf = sqlContext.sql('select * from people')

"""
# Alternatively, on Jupyter notebook you'll do:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()
df = spark.read.json('people.json')"""

Out[22]: "\n# Alternatively, on Jupyter notebook you'll do:\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.appName('Basics').getOrCreate()\ndf = spark.read.json('people.json')"

In [0]:
sdf.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
sdf.printSchema()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [0]:
sdf.columns

Out[25]: ['age', 'name']

In [0]:
sdf.describe()

Out[26]: DataFrame[summary: string, age: string, name: string]

In [0]:
# how to define the schema

from pyspark.sql.types import (StructField, StringType,
                               IntegerType, StructType)

In [0]:
dir(sqlContext)

Out[28]: ['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_conf',
 '_inferSchema',
 '_instantiatedContext',
 '_jsc',
 '_jsqlContext',
 '_jvm',
 '_sc',
 '_ssql_ctx',
 'cacheTable',
 'clearCache',
 'createDataFrame',
 'createExternalTable',
 'dropTempTable',
 'getConf',
 'getOrCreate',
 'newSession',
 'range',
 'read',
 'readStream',
 'registerDataFrameAsTable',
 'registerFunction',
 'registerJavaFunction',
 'setConf',
 'sparkSession',
 'sql',
 'streams',
 'table',
 'tableNames',
 'tables',
 'udf',
 'uncacheTable']

In [0]:
%sql
select * from people

age,name
null,Michael
30,Andy
19,Justin
